In [1]:
from graph_tool import *
from tqdm import tqdm
import csv

In [ ]:
# check if all edges are the same in all samples
d = dict()
with open('../data/REMActivity_1.csv') as file:
    reader = csv.reader(file)
    counter = 0
    for row in tqdm(reader):
        if row[0] != "REMID":
            if d.get(row[1]) != None:
                d[row[1]].add(row[0])
            else:
                d[row[1]] = {row[0]}
        
check = list(d.values())[0]
for k in d.keys():
#     print(k)
    if  len(check.difference(d.get(k))) != 0:
        print(k)

In [2]:
celltype_dict = dict()
with open('../data/sampleToCelltype.csv') as sampleToCelltype:
    reader = csv.reader(sampleToCelltype)
    for row in reader:
        if row[0] != "sampleID":
            celltype_dict[row[0]] = [row[1], row[2]]

In [4]:
rem_dict = dict()
with open('../data/REMActivity_1.csv') as file_1:# TODO: mehrere samples aus celltype hinzufügen, bis jetzt nur für 1 gemacht
    reader_1 = csv.reader(file_1)
    for r in tqdm(reader_1):
        if r[1] == "B_S00DFM11":
            rem_dict[r[0]] = [r[1], r[3]]

FileNotFoundError: [Errno 2] No such file or directory: '../data/REMActivity_1.csv'

In [ ]:
g = Graph()
name = g.new_vp("string")
g.vp.name = name
type = g.new_vp("string")
g.vp.type = type
standDnase1Log2 = g.new_ep("double")
g.edge_properties["standDnase1Log2"] = standDnase1Log2
celltypeID = g.new_ep("string")
g.edge_properties["celltypeID"] = celltypeID
celltype = g.new_ep("string")
g.edge_properties["celltype"] = celltype
sampleID = g.new_ep("string")
g.edge_properties["sampleID"] = sampleID

done_proteins_dict = {}
with open('../data/REMAnnotationModelScore_1.csv') as file:
    reader = csv.reader(file)
    for row in tqdm(reader):
        if row[1] != "start":
            v1 = g.add_vertex()
            g.vp.name[v1] = row[4]
            g.vp.type[v1] = "REM"

            if done_proteins_dict.get(row[3]) == None:
                v2 = g.add_vertex()
                g.vp.name[v2] = row[3]
                g.vp.type[v2] = "Gene"
                done_proteins_dict[row[3]] = int(v2)
            else:
                v2 = g.vertex(done_proteins_dict.get(row[3]))
            
            e = g.add_edge(v1, v2)
            g.ep.standDnase1Log2[e] = rem_dict.get(row[4])[1]
            g.ep.celltypeID[e] = celltype_dict.get(rem_dict.get(row[4])[0])[0]
            g.ep.celltype[e] = celltype_dict.get(rem_dict.get(row[4])[0])[1]
            g.ep.sampleID[e] = rem_dict.get(row[4])[0]


In [ ]:
g.save("../data/graph-tool/specificCelltypeGraph_B_S00DFM11.gt")

In [ ]:
gr = load_graph("../data/graph-tool/specificCelltypeGraph_B_S00DFM11.gt")

In [ ]:
gr.list_properties()
rowCountv = sum(1 for e in gr.vertices())
rowCounte = sum(1 for e in gr.edges())
print("Nodes: "+str(rowCountv))
print("Edges: "+str(rowCounte))

In [ ]:
with open('../data/REMActivity_1.csv') as file:
    reader = csv.reader(file)
    counter = 0
    for row in tqdm(reader):
        print(row)
        counter += 1
        if(counter == 200):
            break